## NCAA March Madness Prediction (Pt. 1)

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

#### Part 1: Data Retrieval
Purpose: Given Raw Data of all Division 1 Basketball Schools from the 2018-2019 season, retrieve all NCAA March Madness participants.

In [2]:
# Loading in Raw Data of School Features
basic = pd.read_csv("data/basic_school_18-19.csv")
opponents = pd.read_csv("data/basic_opps_18-19.csv")
advanced = pd.read_csv("data/adv_school_18-19.csv")
opp_adv = pd.read_csv("data/adv_opps_18-19.csv")

print(basic.shape)
basic.head(3)

(353, 34)


,Rk,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Abilene Christian NCAA,34,27,7,0.794,-1.91,-7.34,14,4,...,457,642,0.712,325,1110,525,297,93,407,635
1,2,Air Force,32,14,18,0.438,-4.28,0.24,8,10,...,341,503,0.678,253,1077,434,154,57,423,543
2,3,Akron,33,17,16,0.515,4.86,1.09,8,10,...,380,539,0.705,312,1204,399,185,106,388,569


In [3]:
# Remove all schools that are not March Madness Tourney participants
basic = basic[(basic["School"].str.contains("NCAA"))]
opponents = opponents[(opponents["School"].str.contains("NCAA"))]
advanced = advanced[(advanced["School"].str.contains("NCAA"))]
opp_adv = opp_adv[(opp_adv["School"].str.contains("NCAA"))]

# Check the above (68 Total Competitors, Same Schools in every Table)
assert basic.shape[0] == 68
assert opponents.shape[0] == 68
assert advanced.shape[0] == 68
assert opp_adv.shape[0] == 68

assert sorted(basic["School"].values) == sorted(opponents["School"].values)
assert sorted(opponents["School"].values) == sorted(advanced["School"].values)
assert sorted(opponents["School"].values) == sorted(opp_adv["School"].values)

In [4]:
# Remove "NCAA" String from School Names
# *WARNING* - Do not run previous block after this one, otherwise will eliminate all rows
def remove_NCAA_str(row):
    return row["School"].replace("NCAA", "").strip()

basic["School"] = basic.apply(remove_NCAA_str, axis=1)
opponents["School"] = opponents.apply(remove_NCAA_str, axis=1)
advanced["School"] = advanced.apply(remove_NCAA_str, axis=1)
opp_adv["School"] = opp_adv.apply(remove_NCAA_str, axis=1)

#### Data Cleaning
Purpose: Begin to Remove Columns that are unnecessary before merge

In [5]:
# Basic Table:
# Dropped: [Rk, G, W-L%, Unnamed: 16, FG%, FT%, 3P%, FT%]
# Reminders: SRS - Simple Rating System, SOS - Strength of Schedule
# W,L.1 - Conference, W,L.2 - Home, W,L.3 - Away
# Tm./Opp. - Points for/against
redundant = ['Rk', 'G', 'W-L%', 'Unnamed: 16', 'FG%', 'FT%', '3P%', 'FT%']
try:
    basic.drop(redundant, axis=1, inplace=True)
except ValueError as err:
    print("Columns Already Removed from Basic Table")

basic.reset_index(drop=True, inplace=True)
basic.head()

,School,W,L,SRS,SOS,W.1,L.1,W.2,L.2,W.3,...,3PA,FT,FTA,ORB,TRB,AST,STL,BLK,TOV,PF
0,Abilene Christian,27,7,-1.91,-7.34,14,4,13,2,10,...,660,457,642,325,1110,525,297,93,407,635
1,Arizona State,23,11,10.28,6.04,12,6,13,3,6,...,714,600,882,399,1351,459,213,109,466,675
2,Auburn,30,10,20.84,10.92,11,7,15,2,4,...,1204,540,760,457,1369,572,369,190,466,731
3,Baylor,20,14,13.38,9.26,10,8,13,5,5,...,803,430,635,446,1280,473,209,159,446,636
4,Belmont,27,6,9.12,-2.60,16,2,13,1,12,...,922,441,598,286,1275,645,220,125,376,509


In [6]:
# Opponents Table
keep = ["School", "FG", "FGA", "3P", "3PA", "FT", "FTA", "ORB", "TRB", "AST", "STL", "BLK", "TOV", "PF"]
try:
    opponents.drop(opponents.columns.difference(keep), axis=1, inplace=True)
except ValueError as err:
    print("Columns already removed from Opponents Table")

# Renaming Columns before Merging with 
opponents.reset_index(drop=True, inplace=True)
opponents.head()

,School,FG,FGA,3P,3PA,FT,FTA,ORB,TRB,AST,STL,BLK,TOV,PF
0,Abilene Christian,753,1737,191,578,464,643,295,1097,353,191,92,560.0,591.0
1,Arizona State,828,2007,292,874,546,774,343,1218,449,221,113,476.0,735.0
2,Auburn,940,2153,323,937,547,774,424,1412,491,226,144,683.0,690.0
3,Baylor,795,1870,246,714,466,682,338,1086,442,202,146,421.0,612.0
4,Belmont,895,2110,253,743,396,569,314,1166,402,200,116,403.0,551.0


In [7]:
# Advanced Statistics Table
original_index = basic.columns.append(pd.Index(redundant))
common_cols = original_index.intersection(advanced.columns)
to_remove = common_cols.drop(["School"])
# print("Dropped: ", to_remove.tolist())

try:
    advanced.drop(to_remove, axis=1, inplace=True)
except ValueError as err:
    print("Columns already removed from Advanced Table")

advanced.reset_index(drop=True, inplace=True)
advanced.head()

,School,Pace,ORtg,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
0,Abilene Christian,67.2,108.6,0.336,0.345,0.565,50.3,58.5,12.9,8.0,0.535,15.5,28.8,0.239
1,Arizona State,72.5,105.4,0.438,0.355,0.543,52.6,51.1,8.5,9.6,0.506,16.1,31.3,0.298
2,Auburn,69.2,114.1,0.312,0.494,0.569,49.2,52.1,13.2,15.6,0.543,14.3,31.6,0.221
3,Baylor,66.6,107.5,0.323,0.408,0.538,54.1,54.4,9.2,13.8,0.512,16.4,37.4,0.219
4,Belmont,74.2,116.2,0.286,0.440,0.603,52.2,61.9,8.9,9.1,0.580,13.7,25.1,0.211


In [8]:
# Advanced Statistics (Opponents) Table
common_cols = original_index.intersection(opp_adv.columns)
to_remove = common_cols.drop(["School"])

try:
    opp_adv.drop(to_remove, axis=1, inplace=True)
except ValueError as err:
    print("Columns already removed from Advanced (Opponents) Table")

opp_adv.reset_index(drop=True, inplace=True)
opp_adv.head()

,School,Pace,ORtg,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
0,Abilene Christian,67.2,93.8,0.370,0.333,0.529,49.7,46.9,8.3,7.4,0.488,21.5,27.3,0.267
1,Arizona State,72.5,99.6,0.386,0.435,0.525,47.4,54.2,8.8,8.7,0.485,16.7,26.5,0.272
2,Auburn,69.2,98.4,0.359,0.435,0.545,50.8,52.2,8.1,11.7,0.512,21.3,31.7,0.254
3,Baylor,66.6,101.3,0.365,0.382,0.525,45.9,55.6,8.9,12.6,0.491,16.1,28.8,0.249
4,Belmont,74.2,98.8,0.270,0.352,0.512,47.8,44.9,8.1,9.9,0.484,14.5,24.1,0.188


#### Merge Tables
Purpose: Construct an extensive feature vector of each school

In [9]:
# Combine features for each school from 4 separate DF's into one dataframe
opponents.columns = [keep[0]] + ["Opp_" + x for x in keep[1:]]
print("Opponent Columns Renamed: ", opponents.columns.tolist())
combined = basic.merge(opponents, how="inner", on="School")
assert combined.shape[0] == 68
assert combined.shape[1] == (basic.shape[1]+opponents.shape[1])-1 # -1 for "School"

if "Opp_3PAr" not in opp_adv.columns.tolist():
    orig = opp_adv.columns.tolist()
    opp_adv.columns = [orig[0]] + ["Opp_" + x for x in orig[1:]]
print("Advanced (Opponents) Columns Renamed: ", opp_adv.columns.tolist())
combined2 = advanced.merge(opp_adv, how="inner", on="School")
assert combined2.shape[0] == 68
assert combined2.shape[1] == (advanced.shape[1]+opp_adv.shape[1])-1

# Ensure only overlapping column is school
assert combined2.columns.intersection(combined.columns) == pd.Index(["School"])

combined3 = combined.merge(combined2, how="inner", on="School")
assert combined3.shape[0] == 68
assert combined3.shape[1] == (combined.shape[1]+combined2.shape[1])-1

combined3.head()

Opponent Columns Renamed:  ['School', 'Opp_FG', 'Opp_FGA', 'Opp_3P', 'Opp_3PA', 'Opp_FT', 'Opp_FTA', 'Opp_ORB', 'Opp_TRB', 'Opp_AST', 'Opp_STL', 'Opp_BLK', 'Opp_TOV', 'Opp_PF']
Advanced (Opponents) Columns Renamed:  ['School', 'Opp_Pace', 'Opp_ORtg', 'Opp_FTr', 'Opp_3PAr', 'Opp_TS%', 'Opp_TRB%', 'Opp_AST%', 'Opp_STL%', 'Opp_BLK%', 'Opp_eFG%', 'Opp_TOV%', 'Opp_ORB%', 'Opp_FT/FGA']


,School,W,L,SRS,SOS,W.1,L.1,W.2,L.2,W.3,...,Opp_3PAr,Opp_TS%,Opp_TRB%,Opp_AST%,Opp_STL%,Opp_BLK%,Opp_eFG%,Opp_TOV%,Opp_ORB%,Opp_FT/FGA
0,Abilene Christian,27,7,-1.91,-7.34,14,4,13,2,10,...,0.333,0.529,49.7,46.9,8.3,7.4,0.488,21.5,27.3,0.267
1,Arizona State,23,11,10.28,6.04,12,6,13,3,6,...,0.435,0.525,47.4,54.2,8.8,8.7,0.485,16.7,26.5,0.272
2,Auburn,30,10,20.84,10.92,11,7,15,2,4,...,0.435,0.545,50.8,52.2,8.1,11.7,0.512,21.3,31.7,0.254
3,Baylor,20,14,13.38,9.26,10,8,13,5,5,...,0.382,0.525,45.9,55.6,8.9,12.6,0.491,16.1,28.8,0.249
4,Belmont,27,6,9.12,-2.60,16,2,13,1,12,...,0.352,0.512,47.8,44.9,8.1,9.9,0.484,14.5,24.1,0.188


#### Saving Results
Purpose: Transfer dataframe values to CSV to be used in next notebook for EDA analysis.
Modularizing the code means chunks of code don't have to be run repeatedly.

In [13]:
combined3.to_csv("data/cleaned_teams.csv", index=False)
print(combined3.columns)
combined3.head()

Index(['School', 'W', 'L', 'SRS', 'SOS', 'W.1', 'L.1', 'W.2', 'L.2', 'W.3',
       'L.3', 'Tm.', 'Opp.', 'MP', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA',
       'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'Opp_FG', 'Opp_FGA',
       'Opp_3P', 'Opp_3PA', 'Opp_FT', 'Opp_FTA', 'Opp_ORB', 'Opp_TRB',
       'Opp_AST', 'Opp_STL', 'Opp_BLK', 'Opp_TOV', 'Opp_PF', 'Pace', 'ORtg',
       'FTr', '3PAr', 'TS%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'eFG%', 'TOV%',
       'ORB%', 'FT/FGA', 'Opp_Pace', 'Opp_ORtg', 'Opp_FTr', 'Opp_3PAr',
       'Opp_TS%', 'Opp_TRB%', 'Opp_AST%', 'Opp_STL%', 'Opp_BLK%', 'Opp_eFG%',
       'Opp_TOV%', 'Opp_ORB%', 'Opp_FT/FGA'],
      dtype='object')


,School,W,L,SRS,SOS,W.1,L.1,W.2,L.2,W.3,...,Opp_3PAr,Opp_TS%,Opp_TRB%,Opp_AST%,Opp_STL%,Opp_BLK%,Opp_eFG%,Opp_TOV%,Opp_ORB%,Opp_FT/FGA
0,Abilene Christian,27,7,-1.91,-7.34,14,4,13,2,10,...,0.333,0.529,49.7,46.9,8.3,7.4,0.488,21.5,27.3,0.267
1,Arizona State,23,11,10.28,6.04,12,6,13,3,6,...,0.435,0.525,47.4,54.2,8.8,8.7,0.485,16.7,26.5,0.272
2,Auburn,30,10,20.84,10.92,11,7,15,2,4,...,0.435,0.545,50.8,52.2,8.1,11.7,0.512,21.3,31.7,0.254
3,Baylor,20,14,13.38,9.26,10,8,13,5,5,...,0.382,0.525,45.9,55.6,8.9,12.6,0.491,16.1,28.8,0.249
4,Belmont,27,6,9.12,-2.60,16,2,13,1,12,...,0.352,0.512,47.8,44.9,8.1,9.9,0.484,14.5,24.1,0.188
